# TOWER LOG

In [1]:
import pandas as pd
from my_utils import *
from tqdm import tqdm
import random
import requests

In [2]:
riot_api_key='RGAPI-96f2dd58-1f36-4e86-a50a-cb61514088fd'

In [3]:
def get_puu_id(summoner_id,riot_api_key):
    url = f'https://kr.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}?api_key={riot_api_key}'
    res = requests.get(url).json()
    puu_id = res['puuid']
    return puu_id

In [4]:
def get_match_ids(puu_id,riot_api_key,page=20):
    url = f'https://asia.api.riotgames.com/lol/match/v5/matches/by-puuid/{puu_id}/ids?type=ranked&start=0&count={page}&api_key={riot_api_key}'
    res = requests.get(url).json()
    return res

In [5]:
def get_matches_timelines(match_ids,riot_api_key):
    match_info_list = []
    for match_id in tqdm(match_ids):
        url1 = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={riot_api_key}'
        url2 = f'https://asia.api.riotgames.com/lol/match/v5/matches/{match_id}/timeline?api_key={riot_api_key}'
        res1 = requests.get(url1).json()
        res2 = requests.get(url2).json()
        match_info_list.append([match_id,res1,res2])
    return match_info_list

In [6]:
def get_rawdata(tier):
    # 원하는 티어를 넣으면 해당 디비전에 속한 사람들 특정 몇명만 뽑는 로직
    summoner_id_list = []
    lst = []
    puu_id_list= []
    divisions = ['I','II','III','IV']
    page = random.randrange(1,51)

    print('get SummonerId')
    for division in tqdm(divisions):
        url = f'https://kr.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/{tier}/{division}?page={page}&api_key={riot_api_key}'
        res = requests.get(url).json()
        #res에 있는 데이터들 중에 랜덤으로 3개를 출력해라. 
        lst += random.sample(res,3)
    summoner_id_list = list(map(lambda x:x['summonerId'],lst))
    
    
    # 위에서 뽑은 사람들의 puu_id를 가져오는 로직
    puu_id_list=[]
    print('get PUUID......')
    for summoner_id in tqdm(summoner_id_list):
        puu_id = get_puu_id(summoner_id,riot_api_key)
        puu_id_list.append(puu_id)
    # puu_id들을 통해 match_id들을 뽑는 로직
    
    print('get match id.....')
    match_id_lst = []
    for p in tqdm(puu_id_list):
        match_ids = get_match_ids(p,riot_api_key,2)
        match_id_lst.extend(match_ids)
        
    # match_id를 통해 match_id / matches / timelines가 들어있는 데이터 프레임 만들기 
    print('get maches&timeline....')
    df_create = get_matches_timelines(match_id_lst, riot_api_key)
    df = pd.DataFrame(df_create,columns = ['match_id','matches','timeline'])
    
    # get_maches_time(ines)

    return df

In [7]:
df = get_rawdata('DIAMOND')

get SummonerId


100%|████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.27it/s]


get PUUID......


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:09<00:00,  1.33it/s]


get match id.....


100%|██████████████████████████████████████████████████████████████████████████████████| 12/12 [00:09<00:00,  1.30it/s]


get maches&timeline....


100%|██████████████████████████████████████████████████████████████████████████████████| 24/24 [00:42<00:00,  1.77s/it]


In [8]:
df.iloc[0].timeline['info']['frames']

[{'events': [{'realTimestamp': 1713870213141,
    'timestamp': 0,
    'type': 'PAUSE_END'}],
  'participantFrames': {'1': {'championStats': {'abilityHaste': 0,
     'abilityPower': 0,
     'armor': 38,
     'armorPen': 0,
     'armorPenPercent': 0,
     'attackDamage': 25,
     'attackSpeed': 100,
     'bonusArmorPenPercent': 0,
     'bonusMagicPenPercent': 0,
     'ccReduction': 0,
     'cooldownReduction': 0,
     'health': 650,
     'healthMax': 650,
     'healthRegen': 0,
     'lifesteal': 0,
     'magicPen': 0,
     'magicPenPercent': 0,
     'magicResist': 32,
     'movementSpeed': 345,
     'omnivamp': 0,
     'physicalVamp': 0,
     'power': 0,
     'powerMax': 0,
     'powerRegen': 0,
     'spellVamp': 0},
    'currentGold': 500,
    'damageStats': {'magicDamageDone': 0,
     'magicDamageDoneToChampions': 0,
     'magicDamageTaken': 0,
     'physicalDamageDone': 0,
     'physicalDamageDoneToChampions': 0,
     'physicalDamageTaken': 0,
     'totalDamageDone': 0,
     'totalDam

In [9]:
tmp_list = list(map(lambda x: x['events'], df.iloc[0].timeline['info']['frames']))

In [10]:
event_list = [element for array in tmp_list for element in array]

In [11]:
event_list

[{'realTimestamp': 1713870213141, 'timestamp': 0, 'type': 'PAUSE_END'},
 {'itemId': 2003,
  'participantId': 9,
  'timestamp': 1695,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 3340,
  'participantId': 9,
  'timestamp': 1862,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 3865,
  'participantId': 5,
  'timestamp': 1964,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 1055,
  'participantId': 9,
  'timestamp': 2265,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 2003,
  'participantId': 5,
  'timestamp': 2332,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 2003,
  'participantId': 5,
  'timestamp': 2532,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 2033,
  'participantId': 6,
  'timestamp': 3774,
  'type': 'ITEM_PURCHASED'},
 {'levelUpType': 'NORMAL',
  'participantId': 5,
  'skillSlot': 1,
  'timestamp': 3909,
  'type': 'SKILL_LEVEL_UP'},
 {'itemId': 3865,
  'participantId': 10,
  'timestamp': 4545,
  'type': 'ITEM_PURCHASED'},
 {'itemId': 2003,
  'participantId': 10,
  'timestamp': 4545,
  'type': 'ITEM_PURCHASED'},

In [12]:
tower_log = list(filter(lambda x: x['type']=='BUILDING_KILL', event_list)) # 특정요소만 뽑아내고 싶을때는 filter쓰면 됨

In [13]:
tower_log

[{'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 4,
  'laneType': 'BOT_LANE',
  'position': {'x': 13866, 'y': 4505},
  'teamId': 200,
  'timestamp': 738323,
  'towerType': 'OUTER_TURRET',
  'type': 'BUILDING_KILL'},
 {'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 0,
  'laneType': 'TOP_LANE',
  'position': {'x': 981, 'y': 10441},
  'teamId': 100,
  'timestamp': 903454,
  'towerType': 'OUTER_TURRET',
  'type': 'BUILDING_KILL'},
 {'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 0,
  'laneType': 'BOT_LANE',
  'position': {'x': 10504, 'y': 1029},
  'teamId': 100,
  'timestamp': 977477,
  'towerType': 'OUTER_TURRET',
  'type': 'BUILDING_KILL'},
 {'assistingParticipantIds': [9, 10],
  'bounty': 0,
  'buildingType': 'TOWER_BUILDING',
  'killerId': 0,
  'laneType': 'MID_LANE',
  'position': {'x': 5846, 'y': 6396},
  'teamId': 100,
  'timestamp': 1115956,
  'towerType': 'OUTER_TURRET',
  'type': 'BUILDING_KILL'},
 {'assistingParticipantIds': [6, 7,

# 어제 문제
- 타워가 부서졌는지 아닌지 여부
- 가장 먼저 타워가 부서진 라인이 어딘지
- 타워가 부서진 라인의 타임스탬프

In [14]:
tower_log[0] # 타워가 부셔진거  + 가장먼저 부셔진 타워

{'bounty': 0,
 'buildingType': 'TOWER_BUILDING',
 'killerId': 4,
 'laneType': 'BOT_LANE',
 'position': {'x': 13866, 'y': 4505},
 'teamId': 200,
 'timestamp': 738323,
 'towerType': 'OUTER_TURRET',
 'type': 'BUILDING_KILL'}

In [15]:
first_tower_lane = tower_log[0]['laneType']
first_tower_team = tower_log[0]['teamId']

In [16]:
first_tower_lane, first_tower_team

('BOT_LANE', 200)

In [17]:
df.iloc[0].matches['info']['participants'][0] # matches에서 첫 번째 사람의 정보

{'allInPings': 0,
 'assistMePings': 0,
 'assists': 3,
 'baronKills': 0,
 'basicPings': 0,
 'bountyLevel': 0,
 'challenges': {'12AssistStreakCount': 0,
  'abilityUses': 209,
  'acesBefore15Minutes': 0,
  'alliedJungleMonsterKills': 0,
  'baronTakedowns': 0,
  'blastConeOppositeOpponentCount': 0,
  'bountyGold': 0,
  'buffsStolen': 0,
  'completeSupportQuestInTime': 0,
  'controlWardsPlaced': 0,
  'damagePerMinute': 388.97255624477214,
  'damageTakenOnTeamPercentage': 0.19863290282643972,
  'dancedWithRiftHerald': 0,
  'deathsByEnemyChamps': 8,
  'dodgeSkillShotsSmallWindow': 0,
  'doubleAces': 0,
  'dragonTakedowns': 0,
  'earlyLaningPhaseGoldExpAdvantage': 0,
  'effectiveHealAndShielding': 0,
  'elderDragonKillsWithOpposingSoul': 0,
  'elderDragonMultikills': 0,
  'enemyChampionImmobilizations': 25,
  'enemyJungleMonsterKills': 0,
  'epicMonsterKillsNearEnemyJungler': 0,
  'epicMonsterKillsWithin30SecondsOfSpawn': 0,
  'epicMonsterSteals': 0,
  'epicMonsterStolenWithoutSmite': 0,
  'fi

In [18]:
df.iloc[0].matches['info']['participants'][0]['teamPosition'] # matches에 6번째 사람도 탑에 팀아이디 200

'TOP'

In [19]:
df.iloc[0].matches['info']['participants'][0]['teamId'] # matches에 6번째 사람도 탑에 팀아이디 200

100

In [20]:
# 포지션을 각라인의 앞글자를 따와서 만들자
def lane_processing(tower_lane, tower_team, lane, team):
    if (tower_lane[0]==lane[0]) and (tower_team==team):
        return 1
    else:
        return 0

In [21]:
def assi_calc(x): # (어제 만든거) 어시스트 계산하는 함수
    try:
        return ','.join(list(map(lambda y: str(y), x['assistingParticipantIds'])))
    except:
        return ''

In [22]:
tmp = df.copy()

In [23]:
col_list = ['matchId', 'gameDuration', 'bans', 'killerId', 'victimId', 'assistId']

In [24]:
df_create = []
for i in range(len(tmp)):
    matchId = tmp.iloc[i].match_id
    match = tmp.iloc[i].matches['info']
    timeline = tmp.iloc[i].timeline['info']
    for j in range(10):
        lst_tmp = []

        # 어제꺼에 좀 추가
        lst_tmp.append(matchId)
        lst_tmp.append(match['gameDuration'])
        lst_tmp.append(match['gameVersion'])
        lst_tmp.append(match['participants'][j]['participantId'])
        lst_tmp.append(match['participants'][j]['championName'])
        lst_tmp.append(match['participants'][j]['teamPosition'])
        lst_tmp.append(match['participants'][j]['teamId'])
        lst_tmp.append(match['participants'][j]['win'])
        lst_tmp.append(match['participants'][j]['kills'])
        lst_tmp.append(match['participants'][j]['deaths'])
        lst_tmp.append(match['participants'][j]['assists'])
        lst_tmp.append(match['participants'][j]['totalDamageDealtToChampions'])
        lst_tmp.append(match['participants'][j]['totalDamageTaken'])

        blue_ban = list(map(lambda x: str(x['championId']), match['teams'][0]['bans']))
        red_ban = list(map(lambda x: str(x['championId']), match['teams'][1]['bans']))
        ban_list = list(set(blue_ban+red_ban)) # 중복된 값이 있을 수도 있으니 set을 사용해서 중복제거
        lst_tmp.append('|'.join(ban_list))

        kill_log = list(map(lambda x: list(filter(lambda z: z['type'] == 'CHAMPION_KILL' , x['events'])) , timeline['frames']))
        kill_log = [element for array in kill_log for element in array]
        k = list(map(lambda x: str(x['killerId']), kill_log))
        d = list(map(lambda x: str(x['victimId']), kill_log))
        a = list(map(lambda x: assi_calc(x), kill_log))
        lst_tmp.append('|'.join(k))
        lst_tmp.append('|'.join(d))
        lst_tmp.append('|'.join(a))

        # 어제 만든거에서 towerlog 추가
        tower_log = list(map(lambda x : list(filter(lambda z:z['type'] == 'BUILDING_KILL' ,x['events'])),timeline['frames']))
        tower_log = [element for array in tower_log for element in array]
        try: 
            ft_tower_lane = tower_log[0]['laneType'] # first_tower_lane
            ft_tower_team = tower_log[0]['teamId']
        except:
            ft_tower_lane = 'n'
            ft_tower_team = 'n'
        ft = lane_processing(ft_tower_lane, ft_tower_team, lst_tmp[5], lst_tmp[6]) # lst_tmp[5] 팀포지션과, lst_tmp[5] 팀아이디
        lst_tmp.append(ft)

        tower_tmp = list(map(lambda x: (x['laneType'],x['teamId'],x['timestamp']) ,tower_log))

        try: 
            laneTower = list(filter(lambda x:(x[0][0] == lst_tmp[5][0] & (x[1]==lst_tmp[6])) ,tower_tmp))[0]
            lane_flag = 1
            laneTowerTime = laneTower[-1]
        except:
            lane_flag = 0
            laneTowerTime = 0

        lst_tmp.append(lane_flag)
        lst_tmp.append(laneTowerTime)

        try:
            g15 = timeline['frames'][14]['participantFrames'][str(j+1)]['totalGold']
        except:
            g15 = 0

        lst_tmp.append(g15)
        df_create.append(lst_tmp)

        col_lst = ['matchId','gameDuration','gameVersion','participantId','championName' ,'teamPosition','teamId','win','kills','deaths', 'assists', 'damageDealt','damageTaken','bans','killerId','victimId','assistId','firstDT','laneTower','LaneTowerTime','G15']

df = pd.DataFrame(df_create, columns=col_lst)

In [25]:
df

,matchId,gameDuration,gameVersion,participantId,championName,teamPosition,teamId,win,kills,deaths,...,damageDealt,damageTaken,bans,killerId,victimId,assistId,firstDT,laneTower,LaneTowerTime,G15
0,KR_7041039473,1398,14.8.576.8838,1,Aatrox,TOP,100,False,1,8,...,9065,20886,910|22|238|72|30|164|-1|236|119,6|4|4|7|2|4|2|4|6|2|4|2|4|9|4|2|6|9|7|6|9|9|7|...,1|10|9|3|8|7|10|9|1|6|10|6|9|5|10|7|2|4|1|1|4|...,"|5|5|8||2,5|4,5|5|7||5|1|5|10|5||7|10|6||7,10|...",0,0,0,4169
1,KR_7041039473,1398,14.8.576.8838,2,LeeSin,JUNGLE,100,False,6,5,...,12084,22596,910|22|238|72|30|164|-1|236|119,6|4|4|7|2|4|2|4|6|2|4|2|4|9|4|2|6|9|7|6|9|9|7|...,1|10|9|3|8|7|10|9|1|6|10|6|9|5|10|7|2|4|1|1|4|...,"|5|5|8||2,5|4,5|5|7||5|1|5|10|5||7|10|6||7,10|...",0,0,0,5529
2,KR_7041039473,1398,14.8.576.8838,3,TwistedFate,MIDDLE,100,False,1,8,...,10950,21958,910|22|238|72|30|164|-1|236|119,6|4|4|7|2|4|2|4|6|2|4|2|4|9|4|2|6|9|7|6|9|9|7|...,1|10|9|3|8|7|10|9|1|6|10|6|9|5|10|7|2|4|1|1|4|...,"|5|5|8||2,5|4,5|5|7||5|1|5|10|5||7|10|6||7,10|...",0,0,0,4106
3,KR_7041039473,1398,14.8.576.8838,4,Kalista,BOTTOM,100,False,13,6,...,21285,18890,910|22|238|72|30|164|-1|236|119,6|4|4|7|2|4|2|4|6|2|4|2|4|9|4|2|6|9|7|6|9|9|7|...,1|10|9|3|8|7|10|9|1|6|10|6|9|5|10|7|2|4|1|1|4|...,"|5|5|8||2,5|4,5|5|7||5|1|5|10|5||7|10|6||7,10|...",0,0,0,8163
4,KR_7041039473,1398,14.8.576.8838,5,Braum,UTILITY,100,False,0,8,...,7895,20818,910|22|238|72|30|164|-1|236|119,6|4|4|7|2|4|2|4|6|2|4|2|4|9|4|2|6|9|7|6|9|9|7|...,1|10|9|3|8|7|10|9|1|6|10|6|9|5|10|7|2|4|1|1|4|...,"|5|5|8||2,5|4,5|5|7||5|1|5|10|5||7|10|6||7,10|...",0,0,0,4326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,KR_7040229658,1546,14.8.576.8838,6,Renekton,TOP,200,False,5,8,...,15868,32754,497|122|72|7|157|221|350|34|35,1|6|3|8|9|1|9|6|3|3|9|4|3|2|9|2|9|5|2|7|2|2|2|...,6|1|8|3|4|6|5|1|6|8|5|10|7|9|5|8|4|7|9|4|7|8|7...,"||||7,10||7,10||2||10|5|4,5|4,5|10|3||3|4,5|9,...",0,0,0,4641
236,KR_7040229658,1546,14.8.576.8838,7,Nunu,JUNGLE,200,False,4,6,...,9100,34166,497|122|72|7|157|221|350|34|35,1|6|3|8|9|1|9|6|3|3|9|4|3|2|9|2|9|5|2|7|2|2|2|...,6|1|8|3|4|6|5|1|6|8|5|10|7|9|5|8|4|7|9|4|7|8|7...,"||||7,10||7,10||2||10|5|4,5|4,5|10|3||3|4,5|9,...",0,0,0,4658
237,KR_7040229658,1546,14.8.576.8838,8,Zed,MIDDLE,200,False,3,8,...,12644,19127,497|122|72|7|157|221|350|34|35,1|6|3|8|9|1|9|6|3|3|9|4|3|2|9|2|9|5|2|7|2|2|2|...,6|1|8|3|4|6|5|1|6|8|5|10|7|9|5|8|4|7|9|4|7|8|7...,"||||7,10||7,10||2||10|5|4,5|4,5|10|3||3|4,5|9,...",1,0,0,4635
238,KR_7040229658,1546,14.8.576.8838,9,Kaisa,BOTTOM,200,False,13,6,...,16849,16257,497|122|72|7|157|221|350|34|35,1|6|3|8|9|1|9|6|3|3|9|4|3|2|9|2|9|5|2|7|2|2|2|...,6|1|8|3|4|6|5|1|6|8|5|10|7|9|5|8|4|7|9|4|7|8|7...,"||||7,10||7,10||2||10|5|4,5|4,5|10|3||3|4,5|9,...",0,0,0,6739


In [26]:
# 머신러닝할때 참고하기 좋은 사이트 (캐글 Kaggle)

In [27]:
# df['kill_point'] = #(kills+assists /같은팀의 총 킬수)

In [28]:
type(df['participantId'][0])

numpy.int64

In [29]:
def kill_point_calc(x):
    ka = x.kills + x.assists
    kill_log = x.killerId.split('|')
    if x.teamId == 100:
        lst = list(filter(lambda x: int(x)<6, kill_log))
    else:
        lst = list(filter(lambda x: int(x)>5, kill_log))

    try: # killog가 없을 수도 있으니
        return round(ka/len(lst), 3)
    except:
        return 0

In [30]:
df['kill_point'] = df.apply(kill_point_calc, axis = 1)

In [31]:
df.sort_values(by='kill_point', ascending=False)

,matchId,gameDuration,gameVersion,participantId,championName,teamPosition,teamId,win,kills,deaths,...,damageTaken,bans,killerId,victimId,assistId,firstDT,laneTower,LaneTowerTime,G15,kill_point
21,KR_7042711579,1520,14.8.576.8838,2,Graves,JUNGLE,100,False,5,7,...,21935,555|72|222|266|166|221|234|64|53,2|4|4|10|8|2|2|8|3|8|3|10|2|8|8|8|5|9|9|2|7|8|...,8|6|9|5|3|7|7|2|6|5|7|5|7|3|2|3|8|4|5|10|5|2|1...,"1,3,4,5|1,2,3,5|1,2,3|9|7|||6|1,2,5|7|2|9|||||...",0,0,0,5961,0.909
36,KR_7042682865,1867,14.8.576.8838,7,Graves,JUNGLE,200,True,7,4,...,22702,555|122|72|238|222|421|84|53,2|7|9|3|4|7|5|7|3|3|3|3|8|4|9|5|3|9|9|2|3|6|7|...,10|5|2|8|10|5|7|2|7|10|7|8|3|10|5|9|6|5|5|10|8...,"4,5|9,10|7,10||5|9,10||8,10|2,4,5|2,4,5|2|||5|...",0,0,0,5360,0.840
118,KR_6999849068,1952,14.6.569.5139,9,Ezreal,BOTTOM,200,True,29,8,...,36623,101|555|78|517|157|104|54|777|53|43,5|6|9|1|5|9|5|9|10|1|10|6|3|9|8|9|9|4|10|9|9|3...,7|5|4|6|7|4|10|5|5|6|4|1|8|4|5|2|5|9|4|2|5|8|1...,"|7|10||4|7,10|4|7,10|9||9|7||7,10|7,9,10|7,10|...",0,0,0,8618,0.837
227,KR_7041886544,1571,14.8.576.8838,8,Taliyah,MIDDLE,200,False,4,3,...,13846,114|72|222|25|76|29|39|221|53,6|7|2|2|4|3|5|2|4|2|8|2|6|4|1|8|2|4|7|8|7|7|5|...,1|5|9|7|10|8|6|7|9|10|3|7|2|8|6|1|10|9|3|5|4|2...,"|9,10|4,5|4|2||2,3|3,5|2,5|4,5|7||7,8|1,2,5|2,...",0,0,0,5626,0.833
226,KR_7041886544,1571,14.8.576.8838,7,Elise,JUNGLE,200,False,5,6,...,22941,114|72|222|25|76|29|39|221|53,6|7|2|2|4|3|5|2|4|2|8|2|6|4|1|8|2|4|7|8|7|7|5|...,1|5|9|7|10|8|6|7|9|10|3|7|2|8|6|1|10|9|3|5|4|2...,"|9,10|4,5|4|2||2,3|3,5|2,5|4,5|7||7,8|1,2,5|2,...",0,0,0,5560,0.833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,KR_6955474886,923,14.3.558.8314,2,Belveth,JUNGLE,100,False,0,2,...,10317,497|6|555|901|77|22|429|57|119,1|9|5|7|6|1|7|7|5|8|9|8|8|7|9|4|3|7|8|6|10|6|0|9,6|4|10|5|2|6|4|5|10|3|2|3|3|5|4|9|8|4|3|4|1|5|8|3,"|10|4|9,10||2|9,10|9,10|4||7,10||7,10|8||||9,1...",0,0,0,4507,0.143
159,KR_7034590561,1497,14.8.576.8838,10,Lux,UTILITY,200,False,0,9,...,18291,102|72|78|157|35|54|421|4|53,4|3|7|7|6|3|2|4|7|7|3|1|2|2|9|7|4|3|3|6|2|2|3|...,10|8|3|2|1|7|6|9|5|4|7|6|9|10|2|2|9|7|10|1|10|...,"5|2|8|6||2|1|5|9,10|9||||4|||5|2|5|||3|||8||6|...",0,0,0,3071,0.143
2,KR_7041039473,1398,14.8.576.8838,3,TwistedFate,MIDDLE,100,False,1,8,...,21958,910|22|238|72|30|164|-1|236|119,6|4|4|7|2|4|2|4|6|2|4|2|4|9|4|2|6|9|7|6|9|9|7|...,1|10|9|3|8|7|10|9|1|6|10|6|9|5|10|7|2|4|1|1|4|...,"|5|5|8||2,5|4,5|5|7||5|1|5|10|5||7|10|6||7,10|...",0,0,0,4106,0.143
92,KR_6955474886,923,14.3.558.8314,3,Hwei,MIDDLE,100,False,1,5,...,11184,497|6|555|901|77|22|429|57|119,1|9|5|7|6|1|7|7|5|8|9|8|8|7|9|4|3|7|8|6|10|6|0|9,6|4|10|5|2|6|4|5|10|3|2|3|3|5|4|9|8|4|3|4|1|5|8|3,"|10|4|9,10||2|9,10|9,10|4||7,10||7,10|8||||9,1...",0,0,0,4997,0.143


####  [gameVersion, matchId, teamPosition, championName, G15] -> rename(CHAMPIONNAME -> enemy_champ, G15 -> enemy_g15)
#### merge를 통해서 blue_team과 위의 컬럼들을 가진 df를 합쳐줌
#### append 사용해서 red의 데이터를 blue 팀 밑에 넣기
#### tmp라는 데이터프레임에 담기

In [32]:
df

,matchId,gameDuration,gameVersion,participantId,championName,teamPosition,teamId,win,kills,deaths,...,damageTaken,bans,killerId,victimId,assistId,firstDT,laneTower,LaneTowerTime,G15,kill_point
0,KR_7041039473,1398,14.8.576.8838,1,Aatrox,TOP,100,False,1,8,...,20886,910|22|238|72|30|164|-1|236|119,6|4|4|7|2|4|2|4|6|2|4|2|4|9|4|2|6|9|7|6|9|9|7|...,1|10|9|3|8|7|10|9|1|6|10|6|9|5|10|7|2|4|1|1|4|...,"|5|5|8||2,5|4,5|5|7||5|1|5|10|5||7|10|6||7,10|...",0,0,0,4169,0.190
1,KR_7041039473,1398,14.8.576.8838,2,LeeSin,JUNGLE,100,False,6,5,...,22596,910|22|238|72|30|164|-1|236|119,6|4|4|7|2|4|2|4|6|2|4|2|4|9|4|2|6|9|7|6|9|9|7|...,1|10|9|3|8|7|10|9|1|6|10|6|9|5|10|7|2|4|1|1|4|...,"|5|5|8||2,5|4,5|5|7||5|1|5|10|5||7|10|6||7,10|...",0,0,0,5529,0.524
2,KR_7041039473,1398,14.8.576.8838,3,TwistedFate,MIDDLE,100,False,1,8,...,21958,910|22|238|72|30|164|-1|236|119,6|4|4|7|2|4|2|4|6|2|4|2|4|9|4|2|6|9|7|6|9|9|7|...,1|10|9|3|8|7|10|9|1|6|10|6|9|5|10|7|2|4|1|1|4|...,"|5|5|8||2,5|4,5|5|7||5|1|5|10|5||7|10|6||7,10|...",0,0,0,4106,0.143
3,KR_7041039473,1398,14.8.576.8838,4,Kalista,BOTTOM,100,False,13,6,...,18890,910|22|238|72|30|164|-1|236|119,6|4|4|7|2|4|2|4|6|2|4|2|4|9|4|2|6|9|7|6|9|9|7|...,1|10|9|3|8|7|10|9|1|6|10|6|9|5|10|7|2|4|1|1|4|...,"|5|5|8||2,5|4,5|5|7||5|1|5|10|5||7|10|6||7,10|...",0,0,0,8163,0.714
4,KR_7041039473,1398,14.8.576.8838,5,Braum,UTILITY,100,False,0,8,...,20818,910|22|238|72|30|164|-1|236|119,6|4|4|7|2|4|2|4|6|2|4|2|4|9|4|2|6|9|7|6|9|9|7|...,1|10|9|3|8|7|10|9|1|6|10|6|9|5|10|7|2|4|1|1|4|...,"|5|5|8||2,5|4,5|5|7||5|1|5|10|5||7|10|6||7,10|...",0,0,0,4326,0.667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,KR_7040229658,1546,14.8.576.8838,6,Renekton,TOP,200,False,5,8,...,32754,497|122|72|7|157|221|350|34|35,1|6|3|8|9|1|9|6|3|3|9|4|3|2|9|2|9|5|2|7|2|2|2|...,6|1|8|3|4|6|5|1|6|8|5|10|7|9|5|8|4|7|9|4|7|8|7...,"||||7,10||7,10||2||10|5|4,5|4,5|10|3||3|4,5|9,...",0,0,0,4641,0.320
236,KR_7040229658,1546,14.8.576.8838,7,Nunu,JUNGLE,200,False,4,6,...,34166,497|122|72|7|157|221|350|34|35,1|6|3|8|9|1|9|6|3|3|9|4|3|2|9|2|9|5|2|7|2|2|2|...,6|1|8|3|4|6|5|1|6|8|5|10|7|9|5|8|4|7|9|4|7|8|7...,"||||7,10||7,10||2||10|5|4,5|4,5|10|3||3|4,5|9,...",0,0,0,4658,0.600
237,KR_7040229658,1546,14.8.576.8838,8,Zed,MIDDLE,200,False,3,8,...,19127,497|122|72|7|157|221|350|34|35,1|6|3|8|9|1|9|6|3|3|9|4|3|2|9|2|9|5|2|7|2|2|2|...,6|1|8|3|4|6|5|1|6|8|5|10|7|9|5|8|4|7|9|4|7|8|7...,"||||7,10||7,10||2||10|5|4,5|4,5|10|3||3|4,5|9,...",1,0,0,4635,0.240
238,KR_7040229658,1546,14.8.576.8838,9,Kaisa,BOTTOM,200,False,13,6,...,16257,497|122|72|7|157|221|350|34|35,1|6|3|8|9|1|9|6|3|3|9|4|3|2|9|2|9|5|2|7|2|2|2|...,6|1|8|3|4|6|5|1|6|8|5|10|7|9|5|8|4|7|9|4|7|8|7...,"||||7,10||7,10||2||10|5|4,5|4,5|10|3||3|4,5|9,...",0,0,0,6739,0.640


In [33]:
db_open()
df = sql_execute('select * from lol_datas')
db_close()

C:\Users\user\게임아이 수업자료\my_utils.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=query, con=db)


오라클 정상적으로 닫힘


In [34]:
df

,GAMEID,GAMEDURATION,GAMEVERSION,RIOTIDGAMENAME,RIOTIDTAGLINE,SUMMONERLEVEL,PARTICIPANTID,CHAMPIONNAME,CHAMPEXPERIENCE,TEAMPOSITION,...,G_16,G_17,G_18,G_19,G_20,G_21,G_22,G_23,G_24,G_25
0,KR_6990811220,1034,14.5.565.1230,BMW,BTC,108,3,Orianna,9673,MIDDLE,...,6375,6996,7025,0,0,0,0,0,0,0
1,KR_6990811220,1034,14.5.565.1230,NPhysique,KR1,151,4,Kaisa,6967,BOTTOM,...,6550,7238,7266,0,0,0,0,0,0,0
2,KR_6990811220,1034,14.5.565.1230,재민좋아,KR1,504,5,Rumble,5388,UTILITY,...,4386,4937,4973,0,0,0,0,0,0,0
3,KR_6990811220,1034,14.5.565.1230,2875226899219520,KR1,51,6,Velkoz,8460,TOP,...,4892,5635,5678,0,0,0,0,0,0,0
4,KR_6990811220,1034,14.5.565.1230,킬각강사,KR1,36,7,Talon,6880,JUNGLE,...,6549,7319,7347,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21415,KR_6972174202,2201,14.4.562.8237,양갈래젖치기,5611,217,6,Yone,17700,TOP,...,7683,8416,8539,8729,9152,9567,10214,10361,10944,11216
21416,KR_6972174202,2201,14.4.562.8237,makyarong,잘하자,49,7,XinZhao,14478,JUNGLE,...,6793,7232,7420,7970,8597,9354,10008,10190,10472,10595
21417,KR_6972174202,2201,14.4.562.8237,오빠 갱 들어간다,KR1,139,8,Kaisa,19070,MIDDLE,...,5675,6140,6450,6739,7120,7473,7958,8933,9370,9492
21418,KR_6972174202,2201,14.4.562.8237,Teddy fan4,KR1,348,9,Zeri,15182,BOTTOM,...,4697,5712,5855,6229,6691,7229,8005,8347,8475,9246


In [35]:
blue_team = df[df.TEAMID == 100]
red_team = df[df.TEAMID == 200]

In [36]:
red_team_tmp = red_team[['GAMEVERSION', 'GAMEID', 'TEAMPOSITION', 'CHAMPIONNAME', 'G_15']].rename(columns = {
    'CHAMPIONNAME' : 'ENEMY_CHAMP','G_15':'ENEMY_G15'
})

In [37]:
red_team_tmp

,GAMEVERSION,GAMEID,TEAMPOSITION,ENEMY_CHAMP,ENEMY_G15
3,14.5.565.1230,KR_6990811220,TOP,Velkoz,4587
4,14.5.565.1230,KR_6990811220,JUNGLE,Talon,5907
5,14.5.565.1230,KR_6990811220,MIDDLE,Leblanc,4963
6,14.5.565.1230,KR_6990811220,BOTTOM,Caitlyn,4825
7,14.5.565.1230,KR_6990811220,UTILITY,Lux,3296
...,...,...,...,...,...
21415,14.4.562.8237,KR_6972174202,TOP,Yone,7383
21416,14.4.562.8237,KR_6972174202,JUNGLE,XinZhao,6475
21417,14.4.562.8237,KR_6972174202,MIDDLE,Kaisa,5276
21418,14.4.562.8237,KR_6972174202,BOTTOM,Zeri,4271


In [38]:
blue_merge_df = pd.merge(blue_team,red_team_tmp,on=['GAMEVERSION','GAMEID','TEAMPOSITION'])

In [39]:
blue_merge_df

,GAMEID,GAMEDURATION,GAMEVERSION,RIOTIDGAMENAME,RIOTIDTAGLINE,SUMMONERLEVEL,PARTICIPANTID,CHAMPIONNAME,CHAMPEXPERIENCE,TEAMPOSITION,...,G_18,G_19,G_20,G_21,G_22,G_23,G_24,G_25,ENEMY_CHAMP,ENEMY_G15
0,KR_6990811220,1034,14.5.565.1230,BMW,BTC,108,3,Orianna,9673,MIDDLE,...,7025,0,0,0,0,0,0,0,Leblanc,4963
1,KR_6990811220,1034,14.5.565.1230,NPhysique,KR1,151,4,Kaisa,6967,BOTTOM,...,7266,0,0,0,0,0,0,0,Caitlyn,4825
2,KR_6990811220,1034,14.5.565.1230,재민좋아,KR1,504,5,Rumble,5388,UTILITY,...,4973,0,0,0,0,0,0,0,Lux,3296
3,KR_7030857833,1290,14.7.571.9528,뿌직탁,KR1,167,1,Zyra,20261,None,...,11300,11798,12906,13385,13798,0,0,0,Varus,12209
4,KR_7030857833,1290,14.7.571.9528,뿌직탁,KR1,167,1,Zyra,20261,None,...,11300,11798,12906,13385,13798,0,0,0,Warwick,10648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21616,KR_6972174202,2201,14.4.562.8237,탑은사랑입니다,KR1,89,1,Jax,19933,TOP,...,6149,6607,6799,7124,7803,8078,8851,9439,Yone,7383
21617,KR_6972174202,2201,14.4.562.8237,봇이터질것같아,T1ooo,106,2,DrMundo,16549,JUNGLE,...,5963,6180,6495,6618,7114,7472,8089,8831,XinZhao,6475
21618,KR_6972174202,2201,14.4.562.8237,KING상민,KR1,715,3,Xerath,19668,MIDDLE,...,6523,6708,6948,7202,7956,8365,8927,9305,Kaisa,5276
21619,KR_6972174202,2201,14.4.562.8237,오시웅2,KR1,37,4,Ezreal,15065,BOTTOM,...,7868,8123,8427,8606,8831,9183,9804,10010,Zeri,4271


In [40]:
blue_team_tmp =blue_team[['GAMEVERSION', 'GAMEID', 'TEAMPOSITION', 'CHAMPIONNAME', 'G_15']].rename(columns = {
    'CHAMPIONNAME' : 'ENEMY_CHAMP','G_15':'ENEMY_G15'
})

In [41]:
red_merge_df =  pd.merge(red_team,blue_team_tmp,on=['GAMEVERSION','GAMEID','TEAMPOSITION'])

In [42]:
red_merge_df

,GAMEID,GAMEDURATION,GAMEVERSION,RIOTIDGAMENAME,RIOTIDTAGLINE,SUMMONERLEVEL,PARTICIPANTID,CHAMPIONNAME,CHAMPEXPERIENCE,TEAMPOSITION,...,G_18,G_19,G_20,G_21,G_22,G_23,G_24,G_25,ENEMY_CHAMP,ENEMY_G15
0,KR_6990811220,1034,14.5.565.1230,2875226899219520,KR1,51,6,Velkoz,8460,TOP,...,5678,0,0,0,0,0,0,0,Kayle,5136
1,KR_6990811220,1034,14.5.565.1230,킬각강사,KR1,36,7,Talon,6880,JUNGLE,...,7347,0,0,0,0,0,0,0,Khazix,5527
2,KR_6990811220,1034,14.5.565.1230,호 민,KR2,45,8,Leblanc,8075,MIDDLE,...,5820,0,0,0,0,0,0,0,Orianna,5906
3,KR_6990811220,1034,14.5.565.1230,모로뛰,KR1,287,9,Caitlyn,6389,BOTTOM,...,5542,0,0,0,0,0,0,0,Kaisa,6184
4,KR_6990811220,1034,14.5.565.1230,뱅미님이다,KR1,36,10,Lux,4573,UTILITY,...,3914,0,0,0,0,0,0,0,Rumble,4200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21616,KR_6972174202,2201,14.4.562.8237,양갈래젖치기,5611,217,6,Yone,17700,TOP,...,8539,8729,9152,9567,10214,10361,10944,11216,Jax,4242
21617,KR_6972174202,2201,14.4.562.8237,makyarong,잘하자,49,7,XinZhao,14478,JUNGLE,...,7420,7970,8597,9354,10008,10190,10472,10595,DrMundo,4941
21618,KR_6972174202,2201,14.4.562.8237,오빠 갱 들어간다,KR1,139,8,Kaisa,19070,MIDDLE,...,6450,6739,7120,7473,7958,8933,9370,9492,Xerath,5239
21619,KR_6972174202,2201,14.4.562.8237,Teddy fan4,KR1,348,9,Zeri,15182,BOTTOM,...,5855,6229,6691,7229,8005,8347,8475,9246,Ezreal,6657


In [43]:
tmp = pd.concat([blue_merge_df,red_merge_df])

In [44]:
tmp

,GAMEID,GAMEDURATION,GAMEVERSION,RIOTIDGAMENAME,RIOTIDTAGLINE,SUMMONERLEVEL,PARTICIPANTID,CHAMPIONNAME,CHAMPEXPERIENCE,TEAMPOSITION,...,G_18,G_19,G_20,G_21,G_22,G_23,G_24,G_25,ENEMY_CHAMP,ENEMY_G15
0,KR_6990811220,1034,14.5.565.1230,BMW,BTC,108,3,Orianna,9673,MIDDLE,...,7025,0,0,0,0,0,0,0,Leblanc,4963
1,KR_6990811220,1034,14.5.565.1230,NPhysique,KR1,151,4,Kaisa,6967,BOTTOM,...,7266,0,0,0,0,0,0,0,Caitlyn,4825
2,KR_6990811220,1034,14.5.565.1230,재민좋아,KR1,504,5,Rumble,5388,UTILITY,...,4973,0,0,0,0,0,0,0,Lux,3296
3,KR_7030857833,1290,14.7.571.9528,뿌직탁,KR1,167,1,Zyra,20261,None,...,11300,11798,12906,13385,13798,0,0,0,Varus,12209
4,KR_7030857833,1290,14.7.571.9528,뿌직탁,KR1,167,1,Zyra,20261,None,...,11300,11798,12906,13385,13798,0,0,0,Warwick,10648
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21616,KR_6972174202,2201,14.4.562.8237,양갈래젖치기,5611,217,6,Yone,17700,TOP,...,8539,8729,9152,9567,10214,10361,10944,11216,Jax,4242
21617,KR_6972174202,2201,14.4.562.8237,makyarong,잘하자,49,7,XinZhao,14478,JUNGLE,...,7420,7970,8597,9354,10008,10190,10472,10595,DrMundo,4941
21618,KR_6972174202,2201,14.4.562.8237,오빠 갱 들어간다,KR1,139,8,Kaisa,19070,MIDDLE,...,6450,6739,7120,7473,7958,8933,9370,9492,Xerath,5239
21619,KR_6972174202,2201,14.4.562.8237,Teddy fan4,KR1,348,9,Zeri,15182,BOTTOM,...,5855,6229,6691,7229,8005,8347,8475,9246,Ezreal,6657


In [45]:
tmp['WIN'] = tmp.apply(lambda x:1 if x.WIN=='True' else 0 ,axis =1)

In [46]:
test = tmp[['GAMEID','CHAMPIONNAME','ENEMY_CHAMP','TEAMPOSITION','WIN']]

In [47]:
test

,GAMEID,CHAMPIONNAME,ENEMY_CHAMP,TEAMPOSITION,WIN
0,KR_6990811220,Orianna,Leblanc,MIDDLE,1
1,KR_6990811220,Kaisa,Caitlyn,BOTTOM,1
2,KR_6990811220,Rumble,Lux,UTILITY,1
3,KR_7030857833,Zyra,Varus,None,1
4,KR_7030857833,Zyra,Warwick,None,1
...,...,...,...,...,...
21616,KR_6972174202,Yone,Jax,TOP,0
21617,KR_6972174202,XinZhao,DrMundo,JUNGLE,0
21618,KR_6972174202,Kaisa,Xerath,MIDDLE,0
21619,KR_6972174202,Zeri,Ezreal,BOTTOM,0


In [48]:
cnt = test[['CHAMPIONNAME','ENEMY_CHAMP','TEAMPOSITION','WIN']].groupby(
    ['CHAMPIONNAME','ENEMY_CHAMP','TEAMPOSITION']).count().rename(columns = {'WIN':'cnt'})

In [49]:
cnt.sort_values(by = 'cnt',ascending=False)[:20]

,,,cnt
CHAMPIONNAME,ENEMY_CHAMP,TEAMPOSITION,
Jinx,Ezreal,BOTTOM,45
Ezreal,Jinx,BOTTOM,45
Caitlyn,Jinx,BOTTOM,42
Jinx,Caitlyn,BOTTOM,42
Caitlyn,Ezreal,BOTTOM,41
Ezreal,Caitlyn,BOTTOM,41
Kaisa,Ezreal,BOTTOM,34
Ezreal,Kaisa,BOTTOM,34
Jhin,Ezreal,BOTTOM,29


In [50]:
win_cnt = test[['CHAMPIONNAME','ENEMY_CHAMP','TEAMPOSITION','WIN']].groupby(
    ['CHAMPIONNAME','ENEMY_CHAMP','TEAMPOSITION']).sum()

In [51]:
result = cnt.join(win_cnt)

In [52]:
result.reset_index(inplace=True)

In [53]:
result['win_per'] = round(result.WIN/ result.cnt,2)

In [54]:
result.sort_values(by = 'cnt',ascending=False)

,CHAMPIONNAME,ENEMY_CHAMP,TEAMPOSITION,cnt,WIN,win_per
2387,Jinx,Ezreal,BOTTOM,45,26,0.58
1409,Ezreal,Jinx,BOTTOM,45,19,0.42
895,Caitlyn,Jinx,BOTTOM,42,21,0.50
2383,Jinx,Caitlyn,BOTTOM,42,21,0.50
891,Caitlyn,Ezreal,BOTTOM,41,26,0.63
...,...,...,...,...,...,...
4347,Orianna,Lux,UTILITY,1,1,1.00
1566,Galio,Ezreal,MIDDLE,1,0,0.00
4344,Orianna,Leblanc,MIDDLE,1,1,1.00
1567,Galio,Hwei,MIDDLE,1,0,0.00


# 해보기
- blue팀 정글포지션 사람들만 꺼내서 [GAMEID,CHAMPIONNAME,WIN].RENAME(CHAMPIONID -> j_champ) \
   (테이블 이름 - blue_jungle)
- blue팀 미드포지션 사람들만 꺼내서 [GAMEID,CHAMPIONNAME,WIN].RENAME(CHAMPIONID -> m_champ)
    (테이블 이름 - blue_middle)
- 둘이 merge
    blue_team
- red팀도 동일한 포지션 동일하게 merge
    red_jungle
    red_top
    red_team

- 위에서 merge한 것을 각각 또 새롭게 테이블을 만들어서 enemy_j_champ, enemy_m_champ로 rename
    blue_tmp = blue_team.rename(enemy_j_champ,enemy_m_champ) 
    red_tmp

- red팀과 blue팀에 enemy_champ들을 merge해서 append
    tmp = merge()

- games와 win횟수를 구하고 win_rate 컬럼 추가한  result 테이블 만들기
    result = 

In [56]:
db_open()
df = sql_execute('select * from lol_datas')
db_close()

C:\Users\user\게임아이 수업자료\my_utils.py:34: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql=query, con=db)


오라클 정상적으로 닫힘


In [57]:
df

,GAMEID,GAMEDURATION,GAMEVERSION,RIOTIDGAMENAME,RIOTIDTAGLINE,SUMMONERLEVEL,PARTICIPANTID,CHAMPIONNAME,CHAMPEXPERIENCE,TEAMPOSITION,...,G_16,G_17,G_18,G_19,G_20,G_21,G_22,G_23,G_24,G_25
0,KR_6990811220,1034,14.5.565.1230,BMW,BTC,108,3,Orianna,9673,MIDDLE,...,6375,6996,7025,0,0,0,0,0,0,0
1,KR_6990811220,1034,14.5.565.1230,NPhysique,KR1,151,4,Kaisa,6967,BOTTOM,...,6550,7238,7266,0,0,0,0,0,0,0
2,KR_6990811220,1034,14.5.565.1230,재민좋아,KR1,504,5,Rumble,5388,UTILITY,...,4386,4937,4973,0,0,0,0,0,0,0
3,KR_6990811220,1034,14.5.565.1230,2875226899219520,KR1,51,6,Velkoz,8460,TOP,...,4892,5635,5678,0,0,0,0,0,0,0
4,KR_6990811220,1034,14.5.565.1230,킬각강사,KR1,36,7,Talon,6880,JUNGLE,...,6549,7319,7347,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21415,KR_6972174202,2201,14.4.562.8237,양갈래젖치기,5611,217,6,Yone,17700,TOP,...,7683,8416,8539,8729,9152,9567,10214,10361,10944,11216
21416,KR_6972174202,2201,14.4.562.8237,makyarong,잘하자,49,7,XinZhao,14478,JUNGLE,...,6793,7232,7420,7970,8597,9354,10008,10190,10472,10595
21417,KR_6972174202,2201,14.4.562.8237,오빠 갱 들어간다,KR1,139,8,Kaisa,19070,MIDDLE,...,5675,6140,6450,6739,7120,7473,7958,8933,9370,9492
21418,KR_6972174202,2201,14.4.562.8237,Teddy fan4,KR1,348,9,Zeri,15182,BOTTOM,...,4697,5712,5855,6229,6691,7229,8005,8347,8475,9246


In [94]:
blue_bot = df[(df.TEAMID == 100) & (df.TEAMPOSITION == 'BOTTOM')][['GAMEID', 'CHAMPIONNAME', 'WIN']].rename(columns={'CHAMPIONNAME':'b_champ'})
blue_sup = df[(df.TEAMID == 100) & (df.TEAMPOSITION == 'UTILITY')][['GAMEID', 'CHAMPIONNAME', 'WIN']].rename(columns={'CHAMPIONNAME':'s_champ'})

In [95]:
blue_result = pd.merge(blue_bot, blue_sup, on=['GAMEID', 'WIN'])

In [96]:
blue_result['WIN'] = blue_result.apply(lambda x:1 if x.WIN == 'True' else 0, axis = 1)

In [97]:
blue_result

,GAMEID,b_champ,WIN,s_champ
0,KR_6990811220,Kaisa,1,Rumble
1,KR_7029062289,Lucian,1,Blitzcrank
2,KR_6958133050,Sivir,1,Xerath
3,KR_7028814319,Jinx,0,Morgana
4,KR_7039823669,Yasuo,1,Shaco
...,...,...,...,...
1588,KR_7042591687,Sivir,0,Xerath
1589,KR_7042539251,Jinx,1,Karma
1590,KR_7037782934,Kaisa,0,Blitzcrank
1591,KR_6980633946,Tristana,1,Pyke


In [98]:
red_bot = df[(df.TEAMID == 200) & (df.TEAMPOSITION == 'BOTTOM')][['GAMEID', 'CHAMPIONNAME', 'WIN']].rename(columns={'CHAMPIONNAME':'b_champ'})
red_sup = df[(df.TEAMID == 200) & (df.TEAMPOSITION == 'UTILITY')][['GAMEID', 'CHAMPIONNAME', 'WIN']].rename(columns={'CHAMPIONNAME':'s_champ'})

In [99]:
red_result = pd.merge(red_bot, red_sup, on=['GAMEID', 'WIN'])

In [100]:
red_result['WIN'] = red_result.apply(lambda x:1 if x.WIN == 'True' else 0, axis = 1)

In [101]:
red_result

,GAMEID,b_champ,WIN,s_champ
0,KR_6990811220,Caitlyn,0,Lux
1,KR_7029062289,Kaisa,0,Morgana
2,KR_6958133050,Ashe,0,Zilean
3,KR_7028814319,Lucian,1,Blitzcrank
4,KR_7039823669,Teemo,0,Yone
...,...,...,...,...
1587,KR_7042539251,Jhin,0,Xerath
1588,KR_7039435745,Ezreal,1,Morgana
1589,KR_7037782934,Yone,1,Alistar
1590,KR_6980633946,Jinx,0,Lux


In [102]:
blue_team_tmp = blue_result[['GAMEID', 'b_champ', 's_champ']].rename(columns = {'b_champ':'enemy_b_champ', 's_champ':'enemy_s_champ'})
red_team_tmp = red_result[['GAMEID', 'b_champ', 's_champ']].rename(columns = {'b_champ':'enemy_b_champ', 's_champ':'enemy_s_champ'})

In [103]:
blue_tmp = pd.merge(blue_result, red_team_tmp, on = ['GAMEID'])
red_tmp = pd.merge(red_result, blue_team_tmp, on = ['GAMEID'])

In [104]:
blue_tmp

,GAMEID,b_champ,WIN,s_champ,enemy_b_champ,enemy_s_champ
0,KR_6990811220,Kaisa,1,Rumble,Caitlyn,Lux
1,KR_7029062289,Lucian,1,Blitzcrank,Kaisa,Morgana
2,KR_6958133050,Sivir,1,Xerath,Ashe,Zilean
3,KR_7028814319,Jinx,0,Morgana,Lucian,Blitzcrank
4,KR_7039823669,Yasuo,1,Shaco,Teemo,Yone
...,...,...,...,...,...,...
1584,KR_7042591687,Sivir,0,Xerath,Ezreal,Pyke
1585,KR_7042539251,Jinx,1,Karma,Jhin,Xerath
1586,KR_7037782934,Kaisa,0,Blitzcrank,Yone,Alistar
1587,KR_6980633946,Tristana,1,Pyke,Jinx,Lux


In [105]:
red_tmp

,GAMEID,b_champ,WIN,s_champ,enemy_b_champ,enemy_s_champ
0,KR_6990811220,Caitlyn,0,Lux,Kaisa,Rumble
1,KR_7029062289,Kaisa,0,Morgana,Lucian,Blitzcrank
2,KR_6958133050,Ashe,0,Zilean,Sivir,Xerath
3,KR_7028814319,Lucian,1,Blitzcrank,Jinx,Morgana
4,KR_7039823669,Teemo,0,Yone,Yasuo,Shaco
...,...,...,...,...,...,...
1584,KR_7042539251,Jhin,0,Xerath,Jinx,Karma
1585,KR_7039435745,Ezreal,1,Morgana,Caitlyn,Hwei
1586,KR_7037782934,Yone,1,Alistar,Kaisa,Blitzcrank
1587,KR_6980633946,Jinx,0,Lux,Tristana,Pyke


In [106]:
tmp = pd.concat([blue_tmp, red_tmp])

In [107]:
tmp

,GAMEID,b_champ,WIN,s_champ,enemy_b_champ,enemy_s_champ
0,KR_6990811220,Kaisa,1,Rumble,Caitlyn,Lux
1,KR_7029062289,Lucian,1,Blitzcrank,Kaisa,Morgana
2,KR_6958133050,Sivir,1,Xerath,Ashe,Zilean
3,KR_7028814319,Jinx,0,Morgana,Lucian,Blitzcrank
4,KR_7039823669,Yasuo,1,Shaco,Teemo,Yone
...,...,...,...,...,...,...
1584,KR_7042539251,Jhin,0,Xerath,Jinx,Karma
1585,KR_7039435745,Ezreal,1,Morgana,Caitlyn,Hwei
1586,KR_7037782934,Yone,1,Alistar,Kaisa,Blitzcrank
1587,KR_6980633946,Jinx,0,Lux,Tristana,Pyke


In [108]:
tmp.drop('GAMEID', axis=1, inplace=True)

In [109]:
games = tmp.groupby(['b_champ', 's_champ','enemy_b_champ','enemy_s_champ']).count().rename(columns = {'WIN':'CNT'})

In [110]:
win = tmp.groupby(['b_champ', 's_champ','enemy_b_champ','enemy_s_champ']).sum() # 총 이긴 횟수

In [111]:
win

WIN
b_champ s_champ enemy_b_champ enemy_s_champ     
Aatrox  Aatrox  Fizz          Fizz             1
                Thresh        Thresh           0
                Zac           Zac              1
        Twitch  Jinx          Yuumi            0
        Yuumi   Teemo         Pyke             0
...                                          ...
Ziggs   Thresh  Caitlyn       Soraka           0
        Varus   Smolder       Rakan            1
        Zyra    Jinx          TahmKench        0
Zilean  Zilean  Nautilus      Nautilus         1
Zyra    Zyra    Akali         Akali            1

[3146 rows x 1 columns]

In [112]:
result = games.join(win)

In [113]:
result

CNT  WIN
b_champ s_champ enemy_b_champ enemy_s_champ          
Aatrox  Aatrox  Fizz          Fizz             1    1
                Thresh        Thresh           1    0
                Zac           Zac              1    1
        Twitch  Jinx          Yuumi            1    0
        Yuumi   Teemo         Pyke             1    0
...                                          ...  ...
Ziggs   Thresh  Caitlyn       Soraka           1    0
        Varus   Smolder       Rakan            1    1
        Zyra    Jinx          TahmKench        1    0
Zilean  Zilean  Nautilus      Nautilus         1    1
Zyra    Zyra    Akali         Akali            1    1

[3146 rows x 2 columns]

In [115]:
result.sort_values(by=['CNT'], ascending = False)

CNT  WIN
b_champ s_champ enemy_b_champ enemy_s_champ          
Jinx    Thresh  Caitlyn       Lux              3    2
Caitlyn Lux     Jinx          Thresh           3    1
                Ezreal        Karma            3    1
Ezreal  Karma   Caitlyn       Lux              3    2
Caitlyn Lux     Lucian        Senna            2    0
...                                          ...  ...
Jhin    Thresh  Brand         Nami             1    0
                Ezreal        Ashe             1    1
                              Zyra             1    0
                Jinx          Mordekaiser      1    0
Zyra    Zyra    Akali         Akali            1    1

[3146 rows x 2 columns]

# Riot Api

In [155]:
url = 'https://ddragon.leagueoflegends.com/cdn/14.8.1/data/ko_KR/champion.json'

In [156]:
res = requests.get(url).json()

In [157]:
res

{'type': 'champion',
 'format': 'standAloneComplex',
 'version': '14.8.1',
 'data': {'Aatrox': {'version': '14.8.1',
   'id': 'Aatrox',
   'key': '266',
   'name': '아트록스',
   'title': '다르킨의 검',
   'blurb': '한때는 공허에 맞서 싸웠던 슈리마의 명예로운 수호자 아트록스와 그의 종족은 결국 공허보다 위험한 존재가 되어 룬테라의 존속을 위협했지만, 교활한 필멸자의 마법에 속아넘어가 패배하게 되었다. 수백 년에 걸친 봉인 끝에, 아트록스는 자신의 정기가 깃든 마법 무기를 휘두르는 어리석은 자들을 타락시키고 육신을 바꾸는 것으로 다시 한번 자유의 길을 찾아내었다. 이제 이전의 잔혹한 모습을 닮은 육체를 차지한 아트록스는 세상의 종말과 오랫동안 기다려온 복수를...',
   'info': {'attack': 8, 'defense': 4, 'magic': 3, 'difficulty': 4},
   'image': {'full': 'Aatrox.png',
    'sprite': 'champion0.png',
    'group': 'champion',
    'x': 0,
    'y': 0,
    'w': 48,
    'h': 48},
   'tags': ['Fighter', 'Tank'],
   'partype': '피의 샘',
   'stats': {'hp': 650,
    'hpperlevel': 114,
    'mp': 0,
    'mpperlevel': 0,
    'movespeed': 345,
    'armor': 38,
    'armorperlevel': 4.45,
    'spellblock': 32,
    'spellblockperlevel': 2.05,
    'attackrange': 175,
    'hpregen': 3,
    'hpregenperlevel': 1,
  

In [158]:
res['data']

{'Aatrox': {'version': '14.8.1',
  'id': 'Aatrox',
  'key': '266',
  'name': '아트록스',
  'title': '다르킨의 검',
  'blurb': '한때는 공허에 맞서 싸웠던 슈리마의 명예로운 수호자 아트록스와 그의 종족은 결국 공허보다 위험한 존재가 되어 룬테라의 존속을 위협했지만, 교활한 필멸자의 마법에 속아넘어가 패배하게 되었다. 수백 년에 걸친 봉인 끝에, 아트록스는 자신의 정기가 깃든 마법 무기를 휘두르는 어리석은 자들을 타락시키고 육신을 바꾸는 것으로 다시 한번 자유의 길을 찾아내었다. 이제 이전의 잔혹한 모습을 닮은 육체를 차지한 아트록스는 세상의 종말과 오랫동안 기다려온 복수를...',
  'info': {'attack': 8, 'defense': 4, 'magic': 3, 'difficulty': 4},
  'image': {'full': 'Aatrox.png',
   'sprite': 'champion0.png',
   'group': 'champion',
   'x': 0,
   'y': 0,
   'w': 48,
   'h': 48},
  'tags': ['Fighter', 'Tank'],
  'partype': '피의 샘',
  'stats': {'hp': 650,
   'hpperlevel': 114,
   'mp': 0,
   'mpperlevel': 0,
   'movespeed': 345,
   'armor': 38,
   'armorperlevel': 4.45,
   'spellblock': 32,
   'spellblockperlevel': 2.05,
   'attackrange': 175,
   'hpregen': 3,
   'hpregenperlevel': 1,
   'mpregen': 0,
   'mpregenperlevel': 0,
   'crit': 0,
   'critperlevel': 0,
   'attackdamage': 60,
   'attackda

In [159]:
res['data'].keys()

dict_keys(['Aatrox', 'Ahri', 'Akali', 'Akshan', 'Alistar', 'Amumu', 'Anivia', 'Annie', 'Aphelios', 'Ashe', 'AurelionSol', 'Azir', 'Bard', 'Belveth', 'Blitzcrank', 'Brand', 'Braum', 'Briar', 'Caitlyn', 'Camille', 'Cassiopeia', 'Chogath', 'Corki', 'Darius', 'Diana', 'Draven', 'DrMundo', 'Ekko', 'Elise', 'Evelynn', 'Ezreal', 'Fiddlesticks', 'Fiora', 'Fizz', 'Galio', 'Gangplank', 'Garen', 'Gnar', 'Gragas', 'Graves', 'Gwen', 'Hecarim', 'Heimerdinger', 'Hwei', 'Illaoi', 'Irelia', 'Ivern', 'Janna', 'JarvanIV', 'Jax', 'Jayce', 'Jhin', 'Jinx', 'Kaisa', 'Kalista', 'Karma', 'Karthus', 'Kassadin', 'Katarina', 'Kayle', 'Kayn', 'Kennen', 'Khazix', 'Kindred', 'Kled', 'KogMaw', 'KSante', 'Leblanc', 'LeeSin', 'Leona', 'Lillia', 'Lissandra', 'Lucian', 'Lulu', 'Lux', 'Malphite', 'Malzahar', 'Maokai', 'MasterYi', 'Milio', 'MissFortune', 'MonkeyKing', 'Mordekaiser', 'Morgana', 'Naafiri', 'Nami', 'Nasus', 'Nautilus', 'Neeko', 'Nidalee', 'Nilah', 'Nocturne', 'Nunu', 'Olaf', 'Orianna', 'Ornn', 'Pantheon', 'Po

In [160]:
res['data']['Ahri']['name']

'아리'

In [161]:
champ_list = list(map(lambda x: [res['data'][x]['name'],res['data'][x]['key'],x], list(res['data'].keys())))

In [162]:
champ_list # 웬만하면 챔피언 키값(숫자)로 등록해두는게 좋음

[['아트록스', '266', 'Aatrox'],
 ['아리', '103', 'Ahri'],
 ['아칼리', '84', 'Akali'],
 ['아크샨', '166', 'Akshan'],
 ['알리스타', '12', 'Alistar'],
 ['아무무', '32', 'Amumu'],
 ['애니비아', '34', 'Anivia'],
 ['애니', '1', 'Annie'],
 ['아펠리오스', '523', 'Aphelios'],
 ['애쉬', '22', 'Ashe'],
 ['아우렐리온 솔', '136', 'AurelionSol'],
 ['아지르', '268', 'Azir'],
 ['바드', '432', 'Bard'],
 ['벨베스', '200', 'Belveth'],
 ['블리츠크랭크', '53', 'Blitzcrank'],
 ['브랜드', '63', 'Brand'],
 ['브라움', '201', 'Braum'],
 ['브라이어', '233', 'Briar'],
 ['케이틀린', '51', 'Caitlyn'],
 ['카밀', '164', 'Camille'],
 ['카시오페아', '69', 'Cassiopeia'],
 ['초가스', '31', 'Chogath'],
 ['코르키', '42', 'Corki'],
 ['다리우스', '122', 'Darius'],
 ['다이애나', '131', 'Diana'],
 ['드레이븐', '119', 'Draven'],
 ['문도 박사', '36', 'DrMundo'],
 ['에코', '245', 'Ekko'],
 ['엘리스', '60', 'Elise'],
 ['이블린', '28', 'Evelynn'],
 ['이즈리얼', '81', 'Ezreal'],
 ['피들스틱', '9', 'Fiddlesticks'],
 ['피오라', '114', 'Fiora'],
 ['피즈', '105', 'Fizz'],
 ['갈리오', '3', 'Galio'],
 ['갱플랭크', '41', 'Gangplank'],
 ['가렌', '86', 'Garen'],
 

In [163]:
champ_df = pd.DataFrame(champ_list, columns=['champ_kr', 'key', 'champ_en'])
champ_df

,champ_kr,key,champ_en
0,아트록스,266,Aatrox
1,아리,103,Ahri
2,아칼리,84,Akali
3,아크샨,166,Akshan
4,알리스타,12,Alistar
...,...,...,...
162,제리,221,Zeri
163,직스,115,Ziggs
164,질리언,26,Zilean
165,조이,142,Zoe


In [172]:
url2 = 'https://ddragon.leagueoflegends.com/cdn/14.8.1/data/ko_KR/item.json'

In [173]:
res2 = requests.get(url2).json()

In [174]:
res2['data']

{'1001': {'name': '장화',
  'description': '<mainText><stats>이동 속도 <attention>25</attention></stats><br><br></mainText>',
  'colloq': ';똥신;boots;speed',
  'plaintext': '이동 속도가 약간 증가합니다.',
  'into': ['3005', '3047', '3117', '3006', '3009', '3020', '3111', '3158'],
  'image': {'full': '1001.png',
   'sprite': 'item0.png',
   'group': 'item',
   'x': 0,
   'y': 0,
   'w': 48,
   'h': 48},
  'gold': {'base': 300, 'purchasable': True, 'total': 300, 'sell': 210},
  'tags': ['Boots'],
  'maps': {'11': True, '12': True, '21': True, '22': False, '30': False},
  'stats': {'FlatMovementSpeedMod': 25}},
 '1004': {'name': '요정의 부적',
  'description': '<mainText><stats>기본 마나 재생 <attention>50%</attention></stats><br><br></mainText>',
  'colloq': ';요부;faerie;charm',
  'plaintext': '마나 재생량이 약간 증가합니다.',
  'into': ['3114', '4642', '3012'],
  'image': {'full': '1004.png',
   'sprite': 'item0.png',
   'group': 'item',
   'x': 48,
   'y': 0,
   'w': 48,
   'h': 48},
  'gold': {'base': 250, 'purchasable': True, 

In [167]:
item_list = list(map(lambda x: [res2['data'][x]['name'],int(x), res2['data'][x]['colloq']], list(res2['data'].keys())))

In [168]:
item_list

[['Boots', 1001, ';'],
 ['Faerie Charm', 1004, ';'],
 ['Rejuvenation Bead', 1006, ';'],
 ["Giant's Belt", 1011, ';'],
 ['Cloak of Agility', 1018, ';'],
 ['Blasting Wand', 1026, ';'],
 ['Sapphire Crystal', 1027, ';blue'],
 ['Ruby Crystal', 1028, ';red'],
 ['Cloth Armor', 1029, ';'],
 ['Chain Vest', 1031, ';'],
 ['Null-Magic Mantle', 1033, ';'],
 ['Emberknife', 1035, ''],
 ['Long Sword', 1036, ';'],
 ['Pickaxe', 1037, ';'],
 ['B. F. Sword', 1038, ';bf'],
 ['Hailblade', 1039, ';jungle;Jungle'],
 ['Obsidian Edge', 1040, ''],
 ['Dagger', 1042, ';'],
 ['Recurve Bow', 1043, ';'],
 ['Amplifying Tome', 1052, ';amptome'],
 ['Vampiric Scepter', 1053, ';'],
 ["Doran's Shield", 1054, ';dshield'],
 ["Doran's Blade", 1055, ';dblade'],
 ["Doran's Ring", 1056, ';dring'],
 ['Negatron Cloak', 1057, ';'],
 ['Needlessly Large Rod', 1058, ';nlr'],
 ['Dark Seal', 1082, ';Noxian'],
 ['Cull', 1083, ';dblade'],
 ['Scorchclaw Pup', 1101, ''],
 ['Gustwalker Hatchling', 1102, ''],
 ['Mosstomper Seedling', 1103, ''

In [169]:
url3 = 'https://ddragon.leagueoflegends.com/cdn/14.8.1/data/ko_KR/champion/Shaco.json'

In [170]:
res3 = requests.get(url3).json()

In [171]:
res3

{'type': 'champion',
 'format': 'standAloneComplex',
 'version': '14.8.1',
 'data': {'Shaco': {'id': 'Shaco',
   'key': '35',
   'name': '샤코',
   'title': '악마 어릿광대',
   'image': {'full': 'Shaco.png',
    'sprite': 'champion3.png',
    'group': 'champion',
    'x': 288,
    'y': 96,
    'w': 48,
    'h': 48},
   'skins': [{'id': '35000', 'num': 0, 'name': 'default', 'chromas': False},
    {'id': '35001', 'num': 1, 'name': '이상한 나라의 모자장수 샤코', 'chromas': False},
    {'id': '35002', 'num': 2, 'name': '궁전 어릿광대 샤코', 'chromas': False},
    {'id': '35003', 'num': 3, 'name': '호두까기 샤코', 'chromas': False},
    {'id': '35004', 'num': 4, 'name': '태엽장치 샤코', 'chromas': False},
    {'id': '35005', 'num': 5, 'name': '뒷골목 샤코', 'chromas': False},
    {'id': '35006', 'num': 6, 'name': '신바람 탈 샤코', 'chromas': False},
    {'id': '35007', 'num': 7, 'name': '와일드 카드 샤코', 'chromas': False},
    {'id': '35008', 'num': 8, 'name': '암흑의 별 샤코', 'chromas': True},
    {'id': '35015', 'num': 15, 'name': '비전 마법사 샤코', 'chr